In [21]:
from pathlib import Path
import pandas as pd
from osgeo import gdal
from netCDF4 import Dataset

In [22]:
# 用于输入固定格式的daily站点观测数据 自动提取出对应数据的点提取值 便于进行评估
# 输入数据表面支持 tif 和 nc格式
# 20240304 新加入了对flt的支持


In [23]:
# 观测数据输入目录
# csv daily
obs_dir = Path(r"D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05")

In [24]:
# 模拟（待评估）数据
# tif/nc 添加对bil的支持
sim_dir = Path(r'G:\anu_originOutput\data1\FLT')

In [25]:
# 最终输出目录
output_dir = Path(r'D:\PROJECTS\PRECIPITATION_CLASSIFY\TEST_s\anu_SplitToCV_231228\ANUSPLIN\output\TEST\temp\anu2000stations_surface_0.1')

In [26]:
# utils
def extract_flt_pixelvalue(dst, lon, lat):
    geotrans1 = dst.GetGeoTransform()

    x = abs(int((lon - geotrans1[0]) / geotrans1[1]))
    y = abs(int((lat - geotrans1[3]) / geotrans1[5]))
    value = dst.ReadAsArray(x, y, 1, 1)

    if value is None:
        return 0
    else:
        return value[0, 0]

def extract_tif_pixelvalue(dst, lon, lat):
    geotrans1 = dst.GetGeoTransform()

    x = abs(int((lon - geotrans1[0]) / geotrans1[1]))
    y = abs(int((lat - geotrans1[3]) / geotrans1[5]))
    value = dst.GetRasterBand(1).ReadAsArray(x, y, 1, 1)
    if value is None:
        return 0
    else:
        return value[0, 0]

def extract_nc_pixelvalue(nc_dst, lon, lat):
    lon_variable = nc_dst.variables['longitude']
    lat_variable = nc_dst.variables['latitude']

    lon_index = abs(lon_variable[:] - lon).argmin()
    lat_index = abs(lat_variable[:] - lat).argmin()

    value = nc_dst.variables['tp'][0, lat_index, lon_index]
    return value


In [ ]:
# processing

# extract all values and merge
temp_df = pd.DataFrame()

for obs_csv in obs_dir.glob('*.csv'):
    print("已读入" + obs_csv.__str__())

    readindf = pd.read_csv(obs_csv)

    # 可选项 tif nc
    extend_name = 'flt'
    readin_sim = sim_dir.joinpath(f'{obs_csv.stem}.{extend_name}')

    if extend_name == 'tif' or extend_name == 'tiff':
        ds1 = gdal.Open(str(readin_sim))
    elif extend_name == 'nc':
        nc_dst = Dataset(readin_sim, 'r')
    elif extend_name == 'flt':
        # flt同样使用gdal读入
        flt_dst = gdal.Open(str(readin_sim))
    else:
        raise Exception('未知后缀名')



    for index, row in readindf.iterrows():
        if extend_name == 'tif' or extend_name == 'tiff':
            extract_value = extract_tif_pixelvalue(ds1, float(row['longitude']), float(row['latitude']))
        elif extend_name == 'nc':
            extract_value = extract_nc_pixelvalue(nc_dst, float(row['longitude']), float(row['latitude']))
        elif extend_name == 'flt':
            extract_value = extract_flt_pixelvalue(flt_dst, float(row['longitude']), float(row['latitude']))
        else:
            raise Exception('未知后缀名')


        # 如果需要对值进行二次处理如转换单位 应该在这里进行
        readindf.at[index, 'extract'] = extract_value

    readindf['date'] = obs_csv.stem
    temp_df = pd.concat([temp_df, readindf])


# extract unique stcd to export
unique_stcd = temp_df['station_code'].unique().tolist()

error_stcd_list = []

for stcd in unique_stcd:
    output_df = temp_df[temp_df['station_code']==stcd]

    if (output_df['extract'] == -9999).any():
        error_stcd_list.append(stcd)
    else:
        output_df.to_csv(output_dir.joinpath(f"{stcd}.csv"))

print('存在-9999的站点列表', error_stcd_list)

已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000101.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000102.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000103.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000104.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000105.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000106.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000107.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000108.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000109.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000110.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000111.csv
已读入D:\CODE\c5_\fast_evaluate_data\observation\split700stations00-05\20000112.csv
已读入D:\CODE\c5_\fast_evaluate